In [13]:
import anthropic
import yaml
import os

import pandas as pd
import numpy as np
import json

# set up Claude

In [14]:
# Use absolute path to the config.yaml file
config_path = "C:/Users/Ryo/OneDrive/Desktop/LLMs/claude_config.yaml"

In [15]:
with open(config_path) as f:
    config_yaml = yaml.load(f, Loader=yaml.FullLoader)
api_key = config_yaml['token']  # Update key name if necessary

client = anthropic.Anthropic(
    api_key=api_key
)

In [16]:
# select model
# claude-3-5-sonnet-20240620
model = "claude-3-5-sonnet-20240620"

# prepare text data

In [17]:
# Load the Excel file (full)
#file_path = "C:/Users/Ryo/OneDrive/Desktop/Master Thesis/study/study1/raw/sample_forFeeding.xlsx"

# Load the Excel file (2nd half only due to RateLimit error)
#file_path = "C:/Users/Ryo/OneDrive/Desktop/Master Thesis/study/study1/raw/sample_forFeeding_for_claude_2nd_half.xlsx"

text_set = pd.read_excel(file_path, sheet_name='text_only') 
sample_set = pd.read_excel(file_path, sheet_name='sample')

In [18]:
text_set.head()

,posts_filtered
0,'I was really close friends with an {MBTI type...
1,"'I love a lot of different music, including in..."
2,'Exactly! :cheers2:|||Same here! So curious....
3,'I recently remade my last.fm. Not that this w...
4,'Fiend!|||...and yet I notice its absence from...


In [19]:
sample_set

,1st choice,2nd choice,3rd choice,Justification for 1st choice (EI),Justification for 1st choice (NS),Justification for 1st choice (TF),Justification for 1st choice (PJ)
0,ENTP,ENFP,INTP,Displays a range of interests and adaptability...,Shows a preference for exploring abstract idea...,Focuses on logical analysis but also values em...,"Demonstrates flexibility and spontaneity, char..."
1,INFP,ISFP,INFJ,Reflects introspection and a preference for so...,Emphasizes emotional depth and abstract expres...,Prioritizes personal values and emotional unde...,Shows a contemplative and open-ended approach ...
2,ENTP,ENTJ,INTJ,"Engages in diverse discussions, suggesting ext...","Prefers theoretical and abstract discussions, ...","Analytical and objective in decision-making, a...","Adaptable and open to new ideas, traits of per..."


# feed into gpt

In [20]:
keys_list = ['1st choice',	'2nd choice',	'3rd choice',	'Justification for 1st choice (EI)',	'Justification for 1st choice (NS)',	'Justification for 1st choice (TF)',	'Justification for 1st choice (PJ)']

## activate a following cell to conduct a pilot test

In [21]:
tmp_df = text_set.head(10)
tmp_df

,posts_filtered
0,'I was really close friends with an {MBTI type...
1,"'I love a lot of different music, including in..."
2,'Exactly! :cheers2:|||Same here! So curious....
3,'I recently remade my last.fm. Not that this w...
4,'Fiend!|||...and yet I notice its absence from...
5,'i do extremely risky things. as long as im no...
6,'Right now it's beer. I'm trying to get the mo...
7,"'Actually, from these results, it looks like y..."
8,'1. Gay Marriage - Sure. 2. Abortion - Yes. 3....
9,'Nice to see another XSTP pilot. I'm working o...


## a full commit

In [22]:
# pilot
#target_df = tmp_df

# a full commit
target_df = text_set

In [23]:
results = []

for index, row in target_df.iterrows():
    print(row['posts_filtered'])
    prompt = f"Analyze the given one group of quotes containing multiple posts from the same speaker (each post is splitted using the symbol '|||') and classify him/her according to the Myers-Briggs Personality Indicator (MBTI).  Provide the top three MBTI predictions with justifications only for the first choice focusing on the four core aspects of MBTI: Extraversion vs. Introversion (E vs I), Sensing vs. Intuition (S vs N), Thinking vs. Feeling (T vs F), and Judging vs. Perceiving (J vs P). Make sure your response is in json format ready to be read using json.loads() method properly and follows the template in {sample_set}. Make sure your output contains all keys from {keys_list} and your answers corresponding to each of the keys.  Do not add any additional text in your response. Here's set of quotes {row['posts_filtered']}"
    
    response = client.messages.create(
        model=model,
        messages=[
            {"role": "user", "content": prompt}
        ],
        temperature=1.0,
        max_tokens=500
    )
    output = response.content[0].text
    print(output)

    try:
        json_data = output.strip('```json\n```')
        try:
            data = json.loads(json_data)
        except json.JSONDecodeError:
            json_data = output.strip('```json\n```  ')
            data = json.loads(json_data)
        results.append(data)
    except json.JSONDecodeError as e:
        print(f"Failed to decode JSON: {e}")
        print("Faulty JSON data:", json_data)  # This will show the problematic part.
        fallback_data = {
            "1st choice": "na",
            "2nd choice": "na",
            "3rd choice": "na",
            "Justification for 1st choice (EI)": "na",
            "Justification for 1st choice (NS)": "na",
            "Justification for 1st choice (TF)": "na",
            "Justification for 1st choice (PJ)": "na"
        }
        results.append(fallback_data)

# Convert results to DataFrame
output_df = pd.DataFrame(results)

output_df.tail(10)

'I was really close friends with an {MBTI type} in college until some stuff tore us apart in our friend group. Years upon years have passed and she made several attempts to fix it, but I was too hurt to let...|||Northern Lights I freaking love your descriptions of this. You have been very helpful. It would seriously be so cool to meet someone so similar to me.        Szartsky based on your profile...|||I either get asked why I'm so angry or told that I need to smile at least once a day. It is beyond annoying.   I'm also pretty even keeled and content majority of the time as long as I'm allowed to...|||jumbotoo  I'm sure there are others out there. We just aren't in the typical spots. I wish more women were similar to me. I might have some girl friends then...|||We went hiking yesterday and it was disheartening to not be able to enjoy the summit completely due to other hikers. Some girl kept babbling about how she hiked all the way up herself and her friends...|||HAHAH omg yes I am a bi

,1st choice,2nd choice,3rd choice,Justification for 1st choice (EI),Justification for 1st choice (NS),Justification for 1st choice (TF),Justification for 1st choice (PJ)
237,INFP,ENFP,ISFP,Reflects introspection and a preference for so...,"Shows a tendency towards abstract thinking, ex...",Demonstrates a strong focus on personal values...,"Exhibits flexibility in thoughts and actions, ..."
238,ENFP,ENTP,INFP,Demonstrates strong extroversion through enthu...,"Exhibits a clear preference for intuition, oft...","Displays a strong feeling preference, often ex...",Demonstrates a perceiving attitude through ada...
239,ENFJ,ESFJ,INFJ,Demonstrates a strong preference for external ...,"Shows a tendency towards intuition, discussing...","Exhibits a clear preference for feeling, with ...","Displays characteristics of judging, such as a..."
240,ENTP,INTP,ENFP,Demonstrates a mix of extraverted and introver...,"Shows strong preference for intuition, particu...","Exhibits more thinking traits, with logical an...","Displays a clear preference for perceiving, wi..."
241,ENFP,INFP,ENFJ,Demonstrates a preference for extraversion thr...,"Shows a strong inclination towards intuition, ...",Demonstrates a clear preference for feeling ov...,Exhibits traits more aligned with perceiving t...
242,ENFP,ENTP,INFP,Demonstrates a preference for external interac...,"Shows a strong inclination towards intuition, ...",Displays a preference for feeling over thinkin...,"Exhibits traits associated with perceiving, su..."
243,ENFP,ESFP,ENTP,Displays extraverted tendencies through social...,"Shows a strong preference for intuition, often...","Exhibits a clear preference for feeling, often...","Demonstrates flexibility, spontaneity, and ope..."
244,ENFP,ENTP,ESFP,Demonstrates extroversion through enthusiastic...,Shows intuition through appreciation of abstra...,Exhibits feeling preference through emotional ...,Displays perceiving traits through spontaneous...
245,INFP,INFJ,ISFP,Reflects introspection and a preference for so...,Shows a tendency towards abstract thinking and...,Demonstrates a strong focus on personal values...,Exhibits flexibility in thinking and openness ...
246,ESFJ,ENFJ,ISFJ,Demonstrates a preference for external interac...,"Shows a tendency towards concrete, practical m...",Exhibits strong emotional awareness and concer...,"Displays a preference for structure, organizat..."


In [24]:
# Save to Excel
# gpt-4o
#output_df.to_excel("C:/Users/Ryo/OneDrive/Desktop/Master Thesis/study/study1/raw/gpt4o_result_raw.xlsx", index=False)
# gpt-3.5
#output_df.to_excel("C:/Users/Ryo/OneDrive/Desktop/Master Thesis/study/study1/raw/gpt35_result_raw.xlsx", index=False)
# claude sonnet
output_df.to_excel("C:/Users/Ryo/OneDrive/Desktop/Master Thesis/study/study1/raw/claude_result_raw.xlsx", index=False)
